In [1]:
import os
import datetime

from vllm.steer_vectors.request import SteerVectorRequest
from vllm import LLM, SamplingParams

from easysteer.steer import StatisticalControlVector

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

/home/meixinyu/anaconda3/envs/easysteer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-12 19:08:16 [__init__.py:244] Automatically detected platform cuda.


In [2]:
model_path = "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"
# vector_path="/home/meixinyu/EasySteer/vectors/thinking_switch_pca_MATH-500.gguf" #change to your path
vector_path="GSM8K.gguf" #change to your path

control_vector = StatisticalControlVector.import_gguf(vector_path)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='pca', directions={0: memmap([ 0.00432168, -0.02199909, -0.02321268, ..., -0.00344263,
        -0.01230501,  0.00480626], shape=(1536,), dtype=float32), 1: memmap([ 0.01987046,  0.03280366,  0.03120032, ..., -0.02368647,
         0.01520148, -0.01653092], shape=(1536,), dtype=float32), 2: memmap([-0.00510607,  0.01795737, -0.03604069, ..., -0.04757744,
         0.04633784, -0.01485594], shape=(1536,), dtype=float32), 3: memmap([-0.01489841, -0.01592376, -0.03618836, ...,  0.0018658 ,
         0.01329398, -0.00049858], shape=(1536,), dtype=float32), 4: memmap([ 0.02282213,  0.00764716, -0.02953233, ..., -0.00735084,
         0.01396071,  0.00707858], shape=(1536,), dtype=float32), 5: memmap([ 0.01595395,  0.06010181, -0.03764697, ..., -0.01724483,
        -0.02025383, -0.04356156], shape=(1536,), dtype=float32), 6: memmap([ 0.01427394,  0.00484354, -0.05836817, ...,  0.01964263,
        -0.01654702, -0.00549133], shape=(1536,), dtype

In [3]:
steer_vector_request_pos = SteerVectorRequest(
            steer_vector_name="thinking_switch_pos",
            steer_vector_id=1,
            steer_vector_local_path=vector_path,
            scale=8.0,
            prefill_trigger_tokens="-1",
            prefill_trigger_positions=[-1],
            generate_trigger_tokens="-1",
            debug=False,
            algorithm='direct'
        )
steer_vector_request_neg = SteerVectorRequest(
            steer_vector_name="thinking_switch_neg",
            steer_vector_id=2,
            steer_vector_local_path=vector_path,
            scale=-8.0,
            prefill_trigger_tokens="-1",
            prefill_trigger_positions=[-1],
            generate_trigger_tokens="-1",
            debug=False,
            algorithm='direct'
        )

In [4]:
sampling_params = SamplingParams(temperature=0.0,max_tokens=4096)
prompt_template = "<|im_start|>user\n%s<|im_end|>\n<|im_start|>assistant\n"
# prompt = "Find the constant term in the expansion of $$\\left(10x^3-\\frac{1}{2x^2}\\right)^{5}$$"
prompt = "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now."
input = prompt_template % prompt

In [5]:
llm = LLM(model=model_path, enable_steer_vector=True, tensor_parallel_size=1)
output_base = llm.generate(
    input,
    sampling_params
)
generated_text_base = output_base[0].outputs[0].text

output_pos = llm.generate(
                input,
                sampling_params,
                steer_vector_request=steer_vector_request_pos
            )
generated_text_pos = output_pos[0].outputs[0].text

output_neg = llm.generate(
                input,
                sampling_params,
                steer_vector_request=steer_vector_request_neg
            )
generated_text_neg = output_neg[0].outputs[0].text

INFO 07-12 19:08:36 [config.py:841] This model supports multiple tasks: {'generate', 'embed', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 07-12 19:08:36 [config.py:1472] Using max model len 32768
WARNING 07-12 19:08:36 [arg_utils.py:1770] --enable-steer-vector is not supported by the V1 Engine. Falling back to V0. 


2025-07-12 19:08:36,719	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 07-12 19:08:36 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None), seed=0, served_model_name=/data/zju-46/she

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.12it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.12it/s]


INFO 07-12 19:08:39 [default_loader.py:272] Loading weights took 0.99 seconds


INFO 07-12 19:08:40 [model_runner.py:1255] Model loading took 2.8876 GiB and 1.147044 seconds
INFO 07-12 19:08:41 [worker.py:295] Memory profiling takes 1.39 seconds
INFO 07-12 19:08:41 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-12 19:08:41 [worker.py:295] model weights take 2.89GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 2.05GiB; the rest of the memory reserved for KV Cache is 37.68GiB.
INFO 07-12 19:08:42 [executor_base.py:115] # cuda blocks: 88185, # CPU blocks: 9362
INFO 07-12 19:08:42 [executor_base.py:120] Maximum concurrency for 32768 tokens per request: 43.06x
INFO 07-12 19:08:45 [model_runner.py:1592] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider dec

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:20<00:00,  1.67it/s]

INFO 07-12 19:09:06 [model_runner.py:1755] Graph capturing finished in 21 secs, took 0.24 GiB
INFO 07-12 19:09:06 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 26.26 seconds



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.81it/s, est. speed input: 85.14 toks/s, output: 72.46 toks/s]


In [6]:
print(generated_text_base)

To determine Allen's age 10 years from now, we first need to find their current ages. We know that Darrell and Allen's ages are in the ratio of 7:11 and their total age now is 162.

Let's denote Darrell's age by \(7x\) and Allen's age by \(11x\). According to the problem, the sum of their ages is 162. Therefore, we can write the equation:
\[
7x + 11x = 162
\]
Combining like terms, we get:
\[
18x = 162
\]
To find \(x\), we divide both sides of the equation by 18:
\[
x = \frac{162}{18} = 9
\]
Now that we have the value of \(x\), we can find Darrell's and Allen's current ages. Darrell's age is:
\[
7x = 7 \times 9 = 63
\]
And Allen's age is:
\[
11x = 11 \times 9 = 99
\]
Next, we need to calculate Allen's age 10 years from now. Adding 10 years to Allen's current age, we get:
\[
99 + 10 = 109
\]
Therefore, Allen's age 10 years from now is \(\boxed{109}\).


In [7]:
print(generated_text_pos)

To calculate Allen's age 10 years from now, we first need to determine their current ages based on the given ratio and total age.

Given:
- The ratio of Darrell's age to Allen's age is 7:11.
- Their total age now is 162.

Let's denote Darrell's age as \(7x\) and Allen's age as \(11x\), where \(x\) is a common multiplier.

The sum of their ages is given by:
\[7x + 11x = 162\]

Combining like terms, we get:
\[18x = 162\]

To find \(x\), we divide both sides by 18:
\[x = \frac{162}{18} = 9\]

Now we can find their current ages:
- Darrell's age: \(7x = 7 \times 9 = 63\)
- Allen's age: \(11x = 11 \times 9 = 99\)

To find Allen's age 10 years from now, we add 10 to his current age:
\[99 + 10 = 109\]

Therefore, Allen's age 10 years from now will be 109 years old.


In [9]:
print(generated_text_neg)

162/18=9
9*11=99

99-10=89

89 is Allen's age 10 years from now.


In [10]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename=f'fractreason_GSM8K_{timestamp}.txt'
# filename=f'../../temp/fractreason_GSM8K_{timestamp}.txt'
with open(filename, 'w', encoding='utf-8') as f:
    f.write("=== Base ===\n")
    f.write(generated_text_base + "\n\n")
    f.write("=== Positive: Slow thinking ===\n")
    f.write(generated_text_pos + "\n\n")
    f.write("=== Negative: Fast thinking ===\n")
    f.write(generated_text_neg + "\n")